In [ ]:
import os
from typing import List, Dict, Any

try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except Exception:
    from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_upstage import UpstageEmbeddings, ChatUpstage

def classify_dispute_type(query: str) -> str:
    """Heuristic dispute type classifier based on keywords."""
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]

    q = query.lower()
    if any(k in q for k in game_keywords):
        return "게임"
    if any(k in q for k in elearning_keywords):
        return "이러닝"
    if any(k in q for k in web_keywords):
        return "웹콘텐츠"
    return "기타"


def build_rag_chain(
    pdf_path: str = "../data/콘텐츠분쟁해결_사례.pdf",
    use_mmr: bool = False,
    k: int = 5,
    temperature: float = 0.0,
) -> Dict[str, Any]:
    """
    Build a complete RAG pipeline:
    - Load PDF
    - Split into chunks with law-friendly separators
    - Embed with Upstage (solar-embedding-1-large)
    - Vector store with FAISS
    - Retriever (similarity or MMR)
    - LLM (solar-pro) with strict prompt for legal advisory
    - RetrievalQA chain returning source documents
    """

    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,          
        chunk_overlap=300,        
        separators=[
            "\n【사건개요】",
            "\n【쟁점사항】",
            "\n【처리경위】",
            "\n【처리결과】",
            "\n■", "\n\n", "\n", ".", " ", ""
        ],
    )
    splits = text_splitter.split_documents(documents)

    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

    vectorstore = FAISS.from_documents(splits, embeddings)

    retriever = vectorstore.as_retriever(
        search_type="mmr" if use_mmr else "similarity",
        search_kwargs={"k": k} if not use_mmr else {"k": k, "fetch_k": max(k * 4, 16)},
    )

    llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=temperature,
    )

    strict_prompt_template = """당신은 콘텐츠 분야 전문 법률 자문사입니다.

다음 정보를 절대적으로 준수하세요:
- 제공된 분쟁사례(context) 안에서만 답변하세요.
- context에 근거가 없거나 불확실하면 답변하지 말고 반드시 "제시된 사례집에서는 확인할 수 없습니다"라고만 말하세요.
- 추정/일반지식/외부자료 인용을 금지합니다.

관련 분쟁사례:
{context}

상담 내용:
{question}

답변 가이드라인:
1. 사례를 근거로 답변하세요.
2. 관련 법령을 명시하세요.
3. 단계별 해결방안을 제시하세요.
4. 유사 사례를 참조하세요.
5. 없는 정보는 "제시된 사례집에서는 확인할 수 없습니다"라고 하세요.

전문 법률 조언:"""

    prompt = PromptTemplate(
        template=strict_prompt_template,
        input_variables=["context", "question"],
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True,
        verbose=False,
    )

    return {
        "loader": loader,
        "splits": splits,
        "embeddings": embeddings,
        "vectorstore": vectorstore,
        "retriever": retriever,
        "llm": llm,
        "prompt": prompt,
        "qa_chain": qa_chain,
    }


INSUFFICIENT_MSG = "제시된 사례집에서는 확인할 수 없습니다"

def safe_answer(qa_chain, question: str, min_chars_in_context: int = 120):
    """
    Enforce the policy:
    1) Retrieve docs. If none or too short context -> return fixed fallback without calling LLM.
    2) Else call the RetrievalQA chain.
    3) Optionally post-check the output (conservative).
    """
    docs = qa_chain.retriever.get_relevant_documents(question)
    total_len = sum(len(getattr(d, "page_content", "")) for d in docs)

    if len(docs) == 0 or total_len < min_chars_in_context:
        return {"result": INSUFFICIENT_MSG, "source_documents": []}

    out = qa_chain.invoke({"query": question})

    text = (out.get("result") or "").strip()
    if not text:
        out["result"] = INSUFFICIENT_MSG

    return out

prompt_template_basic = """당신은 콘텐츠 분야 전문 법률 자문사입니다.

관련 분쟁사례: {context}
상담 내용: {question}

답변 가이드라인:
1. 사례를 근거로 답변하세요.
2. 관련 법령을 명시하세요.
3. 단계별 해결방안을 제시하세요.
4. 유사 사례를 참조하세요.
5. 없는 정보는 "제시된 사례집에서는 확인할 수 없습니다"라고 하세요.

전문 법률 조언:"""

prompt_basic = PromptTemplate(
    template=prompt_template_basic,
    input_variables=["context", "question"],
)


def run_demo_questions(qa_chain, questions: List[str]) -> None:
    """Run demo questions and print answers + simple source trace."""
    for i, q in enumerate(questions, 1):
        print("=" * 80)
        print(f"[Q{i}] {q}")
        result = safe_answer(qa_chain, q)
        print("\n[Answer]\n")
        print(result["result"].strip())

        print("\n[Top Sources]")
        srcs = result.get("source_documents", []) or []
        for j, d in enumerate(srcs, 1):
            meta = getattr(d, "metadata", {}) or {}
            source = meta.get("source", "")
            page = meta.get("page", None)
            if page is not None:
                print(f"  - ({j}) {source} | page {page}")
            else:
                print(f"  - ({j}) {source}")


if __name__ == "__main__":

    assert os.getenv("UPSTAGE_API_KEY"), "Please set UPSTAGE_API_KEY environment variable."

    components = build_rag_chain(
        pdf_path="./data/콘텐츠분쟁해결_사례.pdf", 
        use_mmr=False,   
        k=5,
        temperature=0.0,  
    )
    qa_chain = components["qa_chain"]

    test_questions = [
        "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
        "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
        "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
        "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
        "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?",
    ]

    run_demo_questions(qa_chain, test_questions)


[Q1] 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?


C:\Users\user\AppData\Local\Temp\ipykernel_19800\3929361239.py:153: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = qa_chain.retriever.get_relevant_documents(question)



[Answer]

### 전문 법률 조언  

#### 1. **사례집 내 유사 사례 분석**  
제시된 사례집에서 시스템 오류로 인한 아이템 복구 거부 사례와 관련된 다음과 같은 사례를 참조할 수 있습니다.  

- **2006_시스템 오류로 소멸된 아이템 복구 요구 (1-가-1)**  
  - **쟁점**: 시스템 오류로 인한 아이템 소멸 시 복구 의무 여부  
  - **결과**: 계정 명의자가 아닌 경우 복구 불가 (약관상 현금거래 및 계정 공유 금지 규정 적용).  
  - **시사점**: 계정 명의자 여부와 게임사의 약관이 복구 가능성에 결정적 영향을 미침.  

- **2009_시스템 오류로 인한 손실 아이템 복구 요구 (1-가-1)**  
  - **쟁점**: 시스템 오류 발생 여부 및 게임머니 소실 책임  
  - **결과**: 게임사는 시스템 오류 증거 불충분을 이유로 복구 거부.  
  - **시사점**: 게임사는 오류 발생 사실을 입증할 증거가 없는 경우 책임을 지지 않음.  

- **2006_프로그램 오류로 소멸된 아이템 복구 요구 (4-가-1)**  
  - **쟁점**: 프로그램 오류 인정 시 복구 의무  
  - **결과**: 게임사가 오류를 인정하고 아이템 복구.  
  - **시사점**: 게임사가 오류를 공식적으로 인정할 경우 복구 가능성 있음.  

#### 2. **관련 법령**  
- **민법 제250조(도품·유실물 반환청구권)**  
  - 게임 아이템이 "금전"에 해당하지 않는 경우, 유실물 반환 청구 가능.  
  - 단, 게임사가 아이템 소유권을 약관으로 보유한 경우 적용 제한적일 수 있음.  
- **전자상거래 등에서의 소비자보호에 관한 법률 제17조(결함 있는 재화에 대한 구제)**  
  - 시스템 오류가 "결함"으로 인정될 경우, 소비자는 수리·교환·환급 등을 요구할 수 있음.  

#### 3. **단계별 해결방안**  
**(1) 게임사와의 직접 협상**  
- **증거 수집**: 시스템 오류 발생 시